In [1]:
%pip install transformers
%pip install datasets
%pip install accelerate -U
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset, load_metric
from transformers import TrainingArguments, Trainer
import torch

In [3]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Load the dataset and metric
dataset = load_dataset('glue', 'sst2')
metric = load_metric('glue', 'sst2')

# Split the dataset
train_dataset = dataset['train']
dev_dataset = dataset['validation']
test_dataset = dataset['test']

# Print a description of the dataset
print("Dataset Description: ", train_dataset.description)

# Print the label space
print("Label Space: ", train_dataset.features["label"].names)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

<ipython-input-4-efd8bc38cdc4>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'sst2')


Dataset Description:  GLUE, the General Language Understanding Evaluation benchmark
(https://gluebenchmark.com/) is a collection of resources for training,
evaluating, and analyzing natural language understanding systems.


Label Space:  ['negative', 'positive']


In [5]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
train_dataset

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [30]:
from typing import List
import torch
from transformers import AutoTokenizer

def custom_tokenize(tokenizer: AutoTokenizer, text: str):
    # Tokenize the texts
    result = tokenizer(text, truncation=True, padding=False)
    # Create attention mask with ones on the main diagonal
    attention_mask = torch.eye(len(result["input_ids"]))

    # Update attention mask for the specified neighborhood distance
    distance = 2
    attention_mask[abs(torch.arange(len(attention_mask))[:, None] - torch.arange(len(attention_mask))) <= distance] = 1

    # Set the first row to 1
    attention_mask[0, :] = 1
    # Add the attention mask to the result
    result["attention_mask"] = torch.unsqueeze(attention_mask, 0)
    # Map the labels to the tokenized inputs
    return result

In [34]:
from torch.nn.utils.rnn import pad_sequence
def custom_collate(batch, pad_token_id):
  input_ids = [batch[i]["input_ids"] for i in range(len(batch))]
  attention_mask = [batch[i]["attention_mask"] for i in range(len(batch))]
  token_type_ids = [batch[i]["token_type_ids"] for i in range(len(batch))]
  max_len = max([len(inp) for inp in input_ids])
  input_ids = pad_sequence(input_ids, batch_first=True, padding_value=pad_token_id)
  token_type_ids = pad_sequence(token_type_ids, batch_first=True)
  attention_mask =


In [31]:
# Encode the datasets
train_dataset_dense_attention = train_dataset.map(lambda example: tokenizer(example['sentence'], truncation=True, padding=False), batched=True)
dev_dataset_dense_attention = dev_dataset.map(lambda example: tokenizer(example['sentence'], truncation=True, padding=False), batched=True)
test_dataset_dense_attention = test_dataset.map(lambda example: tokenizer(example['sentence'], truncation=True, padding=False), batched=True)



train_dataset_sparse_attention = train_dataset.map(lambda example: custom_tokenize(tokenizer, example["sentence"]), batched=False)
dev_dataset_sparse_attention = dev_dataset.map(lambda example: custom_tokenize(tokenizer, example["sentence"]), batched=False)
test_dataset_sparse_attention = test_dataset.map(lambda example: custom_tokenize(tokenizer, example["sentence"]), batched=False)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [32]:
train_dataset_sparse_attention[0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0,
 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]]]}

In [10]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    #num_train_epochs=1,            # total number of training steps
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    warmup_ratio=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    fp16=True,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    eval_steps=100,
    max_steps=120,
    load_best_model_at_end=True,
    logging_steps=100
)

# Initialize the trainer
trainer_dense_attention = Trainer(
    model=model,
    tokenizer=tokenizer,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset_dense_attention,         # training dataset
    eval_dataset=dev_dataset_dense_attention,       # evaluation dataset
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)


In [11]:
trainer_dense_attention.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
100,0.385500,0.262433,0.896789


TrainOutput(global_step=120, training_loss=0.3666109959284464, metrics={'train_runtime': 21.2955, 'train_samples_per_second': 360.639, 'train_steps_per_second': 5.635, 'total_flos': 173554629891840.0, 'train_loss': 0.3666109959284464, 'epoch': 0.11})

In [12]:
test_dataset_dense_attention

Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1821
})

In [24]:
test_dataset_dense_attention
trainer_dense_attention.evaluate(test_dataset_dense_attention)

KeyboardInterrupt: ignored

In [33]:
trainer_sparse_attention = Trainer(
    model=model,
    tokenizer=tokenizer,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset_sparse_attention,         # training dataset
    eval_dataset=dev_dataset_sparse_attention,           # evaluation dataset
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
  )

trainer_sparse_attention.train()

ValueError: ignored

In [ ]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [ ]:
# Evaluate the model on the test dataset

evaluation_results = trainer.evaluate(test_dataset)

# Print the evaluation results
print(evaluation_results)

RuntimeError: ignored

In [ ]:
!nvidia-smi

Tue Dec  5 23:14:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 300W |   9936MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.empty_cache()

RuntimeError: ignored

In [ ]:
custom_tokenize(tokenizer, "I like dogs a lot doggo")["attention_mask"]

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0.],
         [0., 1., 1., 1., 1., 1., 0., 0., 0.],
         [0., 0., 1., 1., 1., 1., 1., 0., 0.],
         [0., 0., 0., 1., 1., 1., 1., 1., 0.],
         [0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 1., 1., 1.]]])